In [2]:
!pip install -q datasets transformers accelerate torchaudio librosa

import json
import numpy as np
import torch
from datasets import Dataset, Audio
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

# ✅ Load JSONL manually (avoiding filesystem error)
with open("dataset.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

dataset = Dataset.from_list(data)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

# ✅ Load Whisper processor and model
model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# ✅ Preprocessing: pad or truncate to 3000 frames
def preprocess(batch):
    audio = batch["audio"]
    mel = processor.feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"],
        return_tensors="np"
    )["input_features"][0]  # (80, T)

    desired_len = 3000
    current_len = mel.shape[1]
    if current_len < desired_len:
        mel = np.pad(mel, ((0, 0), (0, desired_len - current_len)), mode="constant")
    else:
        mel = mel[:, :desired_len]

    batch["input_features"] = torch.tensor(mel, dtype=torch.float32)

    # ✅ Truncate labels to 448 tokens
    tokens = processor.tokenizer(batch["transcription"]).input_ids
    batch["labels"] = tokens[:448]
    return batch

dataset = dataset.map(preprocess)

# ✅ Custom data collator
class WhisperDataCollator:
    def __call__(self, features):
        input_features = [torch.tensor(f["input_features"]) if not isinstance(f["input_features"], torch.Tensor) else f["input_features"] for f in features]
        labels = [torch.tensor(f["labels"]) for f in features]

        return {
            "input_features": torch.stack(input_features),
            "labels": torch.nn.utils.rnn.pad_sequence(
                labels, batch_first=True, padding_value=-100
            ),
        }

# ✅ Training args
training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-finetuned",
    per_device_train_batch_size=1,
    learning_rate=1e-5,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_steps=1,
    save_strategy="epoch",
    fp16=False,
)

# ✅ Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=WhisperDataCollator(),
    tokenizer=processor,  # use the full processor here
)

# ✅ Train
trainer.train()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alahmadfahed (alahmadfahed-fahed) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
1,3.398700
2,3.623000
3,2.551500
4,3.189600
5,2.171600
6,2.945700
7,2.846900
8,1.900400
9,2.705500
10,1.821900


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3464: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=10, training_loss=2.715475845336914, metrics={'train_runtime': 1104.9444, 'train_samples_per_second': 0.009, 'train_steps_per_second': 0.009, 'total_flos': 2885854003200000.0, 'train_loss': 2.715475845336914, 'epoch': 5.0})

In [19]:
!pip install -q transformers torchaudio librosa jiwer

import os
import torch
import torchaudio
from jiwer import wer
from transformers import WhisperProcessor, WhisperForConditionalGeneration, AutoProcessor

# Load original processor and fine-tuned model
processor = AutoProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("./whisper-finetuned/checkpoint-10")
model.generation_config.forced_decoder_ids = None
model.eval()

# ✅ Directory where test audio files (.wav) are stored
audio_dir = "tests"
os.makedirs(audio_dir, exist_ok=True)

# ✅ Transcribe each audio file
results = []
for file_name in sorted(os.listdir(audio_dir)):
    if not file_name.endswith(".wav"):
        continue

    file_path = os.path.join(audio_dir, file_name)
    speech_array, sr = torchaudio.load(file_path)
    if sr != 16000:
        speech_array = torchaudio.transforms.Resample(sr, 16000)(speech_array)

    inputs = processor(
        speech_array.squeeze().numpy(),
        sampling_rate=16000,
        return_tensors="pt",
        return_attention_mask=True
    )
    inputs["decoder_input_ids"] = torch.tensor([[50259]])  # 'en' token for Whisper

    # Then generate as usual
    with torch.no_grad():
        predicted_ids = model.generate(
            inputs["input_features"],
            max_length=448
        )
        transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    results.append((file_name, transcription))
    print(f"{file_name} → {transcription}")

output_path = "inference_results.txt"
with open(output_path, "w") as f:
    for fname, text in results:
        f.write(f"{fname}\t{text.strip()}\n")

print(f"\n✅ Transcriptions saved to: {output_path}")


test1.wav →  Do you have any questions? Here, this is our topic again, because afterwards, just imagine, and conclusion, our key findings are
test2.wav →  They are on slide. I'm not sure. And I mean, most of the floors are just to look nice.
test3.wav →  Were you the ones who were watching this? Yes, yes. You have to put your hands over your ears. Okay, bye.

✅ Transcriptions saved to: inference_results.txt


In [20]:
from jiwer import wer

# Load predictions
with open("inference_results.txt", "r") as f:
    pred_dict = dict(line.strip().split("\t", 1) for line in f if line.strip())

# Load ground truth
with open("ground_truth.txt", "r") as f:
    truth_dict = dict(line.strip().split("\t", 1) for line in f if line.strip())

# Compute WER per file
total_wer = 0
for fname in pred_dict:
    pred = pred_dict[fname].lower()
    truth = truth_dict.get(fname, "").lower()
    file_wer = wer(truth, pred)
    total_wer += file_wer
    print(f"{fname}: WER = {file_wer:.3f}")

# Average WER
avg_wer = total_wer / len(pred_dict)
print(f"\nAverage WER: {avg_wer:.3f}")

test1.wav: WER = 0.368
test2.wav: WER = 0.294
test3.wav: WER = 1.750

Average WER: 0.804
